# Primary Key

A primary key is a column or a group of columns used to uniquely identify a row in a table. 
* The column that participates in the primary key is known as the primary key column.
* A table can have zero or one primary key.
* It cannot have more than one primary key.

It is a good practice to add a primary key to every table. When you add a primary key to a table, PostgreSQL creates a **unique B-tree index** on the column or a group of columns used to define the primary key.

Technically, a primary key constraint is the combination of a `NOT NULL` constraint and a `UNIQUE` constraint.
```postgresql
CREATE TABLE table_name (
  column_1 data_type PRIMARY KEY,
  column_2 data_type,
  …
);
```

**Creating a table with a primary key that consists of two columns**

If the primary key consists of more than one column, you can define it using the table constraint:
```postgresql
CREATE TABLE table_name (
  column_1 data_type,
  column_2 data_type,
  column_3 data_type,
  …
  PRIMARY KEY(column_1, column2, ...)
);
```

**Adding a primary key to an existing table**

To add a primary key to an existing table, you use the `ALTER TABLE ... ADD PRIMARY KEY` statement:
```postgresql
ALTER TABLE table_name
ADD PRIMARY KEY (column_1, column_2, ...);
```

**Assign Constraint name**
* If you don’t explicitly specify the name for the primary key constraint, PostgreSQL will assign a default name to the primary key constraint.
* By default, PostgreSQL uses the format **table-name_pkey** as the default name for the primary key constraint.
* To assign a name for the primary key, you can use the `CONSTRAINT` clause as follows:
```postgresql
CONSTRAINT constraint_name
PRIMARY KEY(column_1, column_2,...);
``

**Adding an auto-incremented primary key to an existing table**
```postgresql
ALTER TABLE vendors
ADD COLUMN vendor_id SERIAL PRIMARY KEY;
```

**Drop a primary key**
```postgresql
ALTER TABLE table_name
DROP CONSTRAINT primary_key_constraint; 
```

# Foreign Key Constraint

## Introduction to PostgreSQL Foreign Key Constraint

In PostgreSQL, a **foreign key** is a column or a group of columns in a table that **uniquely identifies a row in another table**.

A foreign key establishes a link between the data in two tables by referencing the **primary key** or a **unique constraint** of the referenced table.
* The table containing a foreign key is referred to as the **referencing table** or **child table**. 
* Conversely, the table referenced by a foreign key is known as the **referenced table** or **parent table**.

The main purpose of foreign keys is to **maintain referential integrity** in a relational database, ensuring that relationships between the parent and child tables are valid.
* For example, a foreign key **prevents the insertion of values** that do not have corresponding values in the **referenced table**.
* Additionally, a foreign key **maintains consistency** by automatically updating or deleting related rows in the child table when changes occur in the parent table.

A table can have multiple foreign keys depending on its relationships with other tables.

## PostgreSQL foreign key constraint syntax

```postgresql
[CONSTRAINT fk_name]
   FOREIGN KEY(fk_columns)
   REFERENCES parent_table(parent_key_columns)
   [ON DELETE delete_action]
   [ON UPDATE update_action]
```
* First, specify the name for the foreign key constraint after the `CONSTRAINT` keyword.
* The `CONSTRAINT` clause is optional. If you omit it, PostgreSQL will assign an auto-generated name.
* Second, specify one or more foreign key columns in parentheses after the `FOREIGN KEY` keywords.
* Third, specify the parent table and parent key columns referenced by the foreign key columns in the `REFERENCES` clause.
* Finally, specify the desired delete and update actions in the `ON DELETE` and `ON UPDATE` clauses.
* Note that if the foreign key constraint does not have the `ON DELETE` and `ON UPDATE` action, they default to `NO ACTION`.

The delete and update actions determine the behaviors when the primary key in the parent table is deleted and updated.
* Since the primary key is rarely updated, the `ON UPDATE` action is infrequently used in practice.
* We'll focus on the `ON DELETE` action.

PostgreSQL supports the following actions:
* SET NULL
* SET DEFAULT
* RESTRICT
* NO ACTION
* CASCADE

## Foreign Key Example: NO ACTION

```postgresql
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS contacts;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
        REFERENCES customers(customer_id)
); 
```

**Inserts data into the `customers` and `contacts` tables**:
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com'); 
```

**Delete the customer with ID 1 from the `customers` table**:
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE NO ACTION`, PostgreSQL issues a constraint violation because the referencing rows of the customer ID 1 still exist in the `contacts` table:
```postgresql
ERROR:  update or delete on table "customers" violates foreign key constraint "fk_customer" on table "contacts".
DETAIL:  Key (customer_id)=(1) is still referenced from table "contacts".
SQL state: 23503
```

* The `RESTRICT` action is similar to the `NO ACTION`. 
* The difference only arises when you define the foreign key constraint as `DEFERRABLE` with an `INITIALLY DEFERRED` or `INITIALLY IMMEDIATE` mode.


## Foreign Key Example: SET NULL

The `SET NULL` automatically sets `NULL` to the foreign key columns in the referencing rows of the child table when the referenced rows in the parent table are deleted.

```postgresql
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
	  REFERENCES customers(customer_id)
	  ON DELETE SET NULL
);
```

**Insert data into the `customers` and `contacts` tables**:
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com');
```

**Delete the customer with id 1 from the `customers` table:**
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE SET NULL` action, the referencing rows in the contacts table are set to `NULL`.

## Foreign Key Example: CASCADE

* The `ON DELETE CASCADE` automatically deletes all the referencing rows in the child table when the referenced rows in the parent table are deleted.
* In practice, the `ON DELETE CASCADE` is the most commonly used option.
```postgresql
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
	  REFERENCES customers(customer_id)
	  ON DELETE CASCADE
); 
```

**Insert data into the `customers` and `contacts` tables:**
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com');
```

**Deletes the customer ID 1**:
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE CASCADE` action, all the referencing rows in the contacts table are automatically deleted.

## Foreign Key Example: SET DEFAULT

The `ON DELETE SET DEFAULT` sets the default value to the foreign key column of the referencing rows in the child table when the referenced rows from the parent table are deleted.

## Add a foreign key constraint to an existing table

```postgresql
ALTER TABLE child_table
ADD CONSTRAINT constraint_name
FOREIGN KEY (fk_columns)
REFERENCES parent_table (parent_key_columns);
```

When adding a foreign key constraint with the `ON DELETE CASCADE` option to an existing table, you need to follow these steps:

First, drop the existing foreign key constraint:
```postgresql
ALTER TABLE child_table
DROP CONSTRAINT constraint_fkey;
```

Second, add a new foreign key constraint with `ON DELETE CASCADE` action:
```postgresql
ALTER TABLE child_table
ADD CONSTRAINT constraint_fk
FOREIGN KEY (fk_columns)
REFERENCES parent_table(parent_key_columns)
ON DELETE CASCADE;
```

# CHECK constraints

In PostgreSQL, a `CHECK` constraint ensures that values in a column or a group of columns meet a specific condition.
* A check constraint allows you to enforce data integrity rules at the database level.
* A check constraint uses a boolean expression to evaluate the values, ensuring that only valid data is inserted or updated in a table.

## Creating CHECK constraints

```postgresql
CREATE TABLE table_name(
   column1 datatype,
   ...,
   CONSTRAINT constraint_name CHECK(condition)
); 
```
* The `CONSTRAINT` keyword is optional.
* If you omit it, PostgreSQL will automatically generate a name for the `CHECK` constraint.

If the `CHECK` constraint involves only one column, you can define it as a column constraint like this:
```postgresql
CREATE TABLE table_name(
   column1 datatype,
   column1 datatype CHECK(condition),
   ...,
); 
```

By default, PostgreSQL assigns a name to a `CHECK` constraint using the following format:
```postgresql
{table}_{column}_check
```


## Defining PostgreSQL CHECK constraint for a new table

```postgresql
CREATE TABLE employees (
  id SERIAL PRIMARY KEY,
  first_name VARCHAR (50) NOT NULL,
  last_name VARCHAR (50) NOT NULL,
  birth_date DATE NOT NULL,
  joined_date DATE NOT NULL,
  salary numeric CHECK(salary > 0)
); 
```

Here, the **employees** table has one `CHECK` constraint that enforces the values in the **salary column greater than zero**.

**Insert a new row with a negative salary into the employees table**:
```postgresql
INSERT INTO employees (first_name, last_name, birth_date, joined_date, salary)
VALUES ('John', 'Doe', '1972-01-01', '2015-07-01', -100000); 
```

**Error**:
```postgresql
ERROR:  new row for relation "employees" violates check constraint "employees_salary_check"
DETAIL:  Failing row contains (1, John, Doe, 1972-01-01, 2015-07-01, -100000).
```

The insert fails because the `CHECK` constraint on the salary column accepts only positive values.

## Using functions in CHECK constraints

**SYNTAX**:
```postgresql
ALTER TABLE table_name
ADD CONSTRAINT constraint_name CHECK (condition);
```

**EXAMPLE**:
```postgresql
ALTER TABLE employees
ADD CONSTRAINT first_name_check
CHECK ( LENGTH(TRIM(first_name)) >= 3); 
```

The following statement will fail because it attempts to insert a row into the employees table with the first name that has 2 characters:
```postgresql
INSERT INTO employees (first_name, last_name, birth_date, joined_date, salary)
VALUES ('Ab', 'Doe', '1990-01-01', '2008-01-01', 100000);
```

**Error**:
```postgresql
ERROR:  new row for relation "employees" violates check constraint "first_name_check"
DETAIL:  Failing row contains (4, Ab, Doe, 1990-01-01, 2008-01-01, 100000).
```

## Removing a CHECK constraint example

**SYNTAX**:
```postgresql
ALTER TABLE table_name
DROP CONSTRAINT constraint_name;
```

**EXAMPLE**:
```postgresql
ALTER TABLE employees
DROP CONSTRAINT joined_date_check; 
```

# UNIQUE constraint

* Sometimes, you want to ensure that values stored in a column or a group of columns are unique across the whole table, such as email addresses or usernames.
* PostgreSQL provides you with the `UNIQUE` constraint that maintains the uniqueness of the data correctly.
* When a `UNIQUE` constraint is in place, every time you insert a new row, it checks if the value is already in the table.
* It rejects the change and issues an error if the value already exists.
* The same process is carried out for updating existing data.
* When you add a `UNIQUE` constraint to a column or a group of columns, PostgreSQL will automatically create a unique index on the column or the group of columns.

**SYNTAX**:
```postgresql
CREATE TABLE table (
    c1 data_type UNIQUE,
    c2 data_type,
    c3 data_type
); 
```

> Use the `UNIQUE` constraints to enforce values stored in a column or a group of columns unique across rows within the same table.

## Creating a UNIQUE constraint on multiple columns

```postgresql
CREATE TABLE table (
    c1 data_type,
    c2 data_type,
    c3 data_type,
    UNIQUE (c2, c3)
); 
```

## Adding unique constraints using a unique index

**First, suppose you have a table named equipment**:
```postgresql
CREATE TABLE equipment (
  id SERIAL PRIMARY KEY,
  name VARCHAR (50) NOT NULL,
  equip_id VARCHAR (16) NOT NULL
); 
```

**Second, create a unique index based on the `equip_id` column**
```postgresql
CREATE UNIQUE INDEX CONCURRENTLY equipment_equip_id
ON equipment (equip_id); 
```

**Third, add a unique constraint to the equipment table using the `equipment_equip_id` index.**
```postgresql
ALTER TABLE equipment
ADD CONSTRAINT unique_equip_id
UNIQUE USING INDEX equipment_equip_id; 
```

Notice that the `ALTER TABLE` statement acquires an exclusive lock on the table. If you have any pending transactions, it will wait for all transactions to complete before changing the table. Therefore, you should check the `pg_stat_activity` table to see the current pending transactions that are ongoing using the following query:
```postgresql
SELECT
  datid,
  datname,
  usename,
  state
FROM
  pg_stat_activity;
```

* You should look at the result to find the `state` column with the value `idle in the transaction`. 
* Those are the transactions that are still pending to complete.

# Not-Null Constraint

## Introduction to NULL

* In the database world, `NULL` represents unknown or missing information. 
* NULL is not the same as an empty string or the number zero.

Suppose you need to insert the email address of a contact into a table. You can request his or her email address.
* However, if you don't know whether the contact has an email address or not, you can insert `NULL` into the email address column.
* In this case, `NULL` indicates that the email address is unknown at the recording time.

**`NULL` is very special.**
* It does not equal anything, even itself.
* The expression `NULL = NULL` returns `NULL` because it makes sense that two unknown values should not be equal.
* To check if a value is `NULL` or not, you use the `IS NULL` boolean operator. 

For example, the following expression returns true if the value in the email address is `NULL`.
```postgresql
email_address IS NULL
```

The `IS NOT NULL` operator negates the result of the `IS NULL` operator.

## PostgreSQL NOT NULL constraints

To control whether a column can accept `NULL`, you use the `NOT NULL` constraint:
```postgresql
CREATE TABLE table_name(
   ...
   column_name data_type NOT NULL,
   ...
);
```

If a column has a `NOT NULL` constraint, any attempt to insert or update `NULL` in the column **will result in an error**.

## Declaring NOT NULL columns

```postgresql
CREATE TABLE invoices(
  id SERIAL PRIMARY KEY,
  product_id INT NOT NULL,
  qty numeric NOT NULL CHECK(qty > 0),
  net_price numeric CHECK(net_price > 0)
);
```

* If you use `NULL` instead of `NOT NULL`, the column will accept both `NULL` and `non-NULL` values.
* If you don't explicitly specify `NULL` or `NOT NULL`, it will accept `NULL` by default.


## Adding NOT NULL Constraints to existing columns

```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name_1 SET NOT NULL,
ALTER COLUMN column_name_2 SET NOT NULL,
...; 
```

## The special case of NOT NULL constraint

Besides the `NOT NULL` constraint, you can use a `CHECK` constraint to force a column to accept **non-NULL** values. 

The `NOT NULL` constraint is equivalent to the following `CHECK` constraint:
```postgresql
CHECK(column IS NOT NULL)
```

* This is useful because sometimes you may want either column **a** or **b** to be not **null**, but not both.
* For example, you may want either the **username** or **email** column of the user tables to be not null or empty.

In this case, you can use the `CHECK` constraint as follows:
```postgresql
CREATE TABLE users (
  id serial PRIMARY KEY,
  username VARCHAR (50),
  password VARCHAR (50),
  email VARCHAR (50),
  CONSTRAINT username_email_notnull CHECK (
    NOT (
      (username IS NULL OR username = '')
      AND 
      (email IS NULL OR email = '')
    )
  )
);
```

# DEFAULT Value

## Defining the DEFAULT value for a column of a new table

```postgresql
CREATE TABLE table_name(
    column1 type,
    column2 type DEFAULT default_value/expression,
    column3 type,
    ...
); 
```
* If you don’t specify the `DEFAULT` constraint for the column, its default value is `NULL`.
* This often makes sense because `NULL` represents unknown data.
* The default value can be a literal value such as a number, a string, a JSON object, etc.
* Additionally, it can be an **expression** that will be evaluated when the default value is inserted into the table.

**Use the `DEFAULT` constraint to define a default value for a table column.**
```postgresql
INSERT INTO table_name(column1, colum3)
VALUES(value1, value2);
```

**Use the `DEFAULT` keyword to explicitly use the default value specified in the `DEFAULT` constraint in the `INSERT` statement.**
```postgresql
INSERT INTO table_name(column1, column2, colum3)
VALUES(value1,DEFAULT,value2);
```


## Defining the DEFAULT value for a column of an existing table

```postgresql
ALTER TABLE table_name
ALTER COLUMN column2
SET DEFAULT default_value;
```

## Removing the DEFAULT value from a column

```postgresql
ALTER TABLE table_name
ALTER COLUMN column2
DROP DEFAULT;
```

## PostgreSQL default value examples

**Basic PostgreSQL default value examples**
```postgresql
CREATE TABLE products(
   id SERIAL PRIMARY KEY,
   name VARCHAR(255) NOT NULL,
   price DECIMAL(19,2) NOT NULL DEFAULT 0
);
```

**Using DEFAULT constraint with TIMESTAMP columns**
```postgresql
CREATE TABLE logs(
   id SERIAL PRIMARY KEY,
   message TEXT NOT NULL,
   created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

**Using DEFAULT constraint with JSONB type**
```postgresql
CREATE TABLE settings(
   id SERIAL PRIMARY KEY,
   name VARCHAR(50) NOT NULL,
   configuration JSONB DEFAULT '{}'
);
```